# 读取数据

In [135]:
import pymongo
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
client = pymongo.MongoClient('localhost',27017)
db  = client['Ziru_housing']
table = db['Shanghai_Renting']
data = pd.DataFrame(list(table.find()))
del data['_id']

In [2]:
data.head()

,area,district,metro,metro_distance,price,square,title,type
0,42 ㎡06/6层2室1厅,徐汇,距3号线龙漕路站173米,173,4830,42,整租 · 上缝小区2居室-南,整租
1,68.65 ㎡06/6层2室1厅,徐汇,距11号线龙耀路站757米,757,5130,68.65,整租 · 滨江小区2居室-南,整租
2,30.37 ㎡06/6层1室1厅,徐汇,距4号线上海体育馆站236米,236,4690,30.37,整租 · 中原新村1居室-南,整租
3,49.15 ㎡01/6层2室1厅,徐汇,距3号线石龙路站243米,243,4790,49.15,整租 · 东泉小区2居室-南,整租
4,33.17 ㎡04/6层1室1厅,徐汇,距4号线东安路站488米,488,4490,33.17,整租 · 斜土路2200弄1居室-南,整租


In [3]:
data.shape
#data[data['square']=='约13']

(14652, 8)

---

---

# 正则提取float

In [4]:
import re
def square(num):
    # 取薪资均值----------------
    pattern = re.compile('[1-9]\d*.\d*|0.\d*[1-9]\d*')
    temp = num
    res = re.findall(pattern, temp)
    result = 0
    for i in res:
        result = float(i)
    # 函数返回值
    return result

square_list = []
metro_list = []
for i in range(0,14652):
    square_num = square(data['square'][i])
    metro_num =  square(data['metro_distance'][i])
    square_list.append(square_num)
    metro_list.append(metro_num)
#print(square_list)

In [5]:
sq = Series(square_list)
md = Series(metro_list)
data.insert(6,'S',sq)
data.insert(4,'metro_num',md)

In [6]:
data.head()

,area,district,metro,metro_distance,metro_num,price,square,S,title,type
0,42 ㎡06/6层2室1厅,徐汇,距3号线龙漕路站173米,173,173.0,4830,42,42.00,整租 · 上缝小区2居室-南,整租
1,68.65 ㎡06/6层2室1厅,徐汇,距11号线龙耀路站757米,757,757.0,5130,68.65,68.65,整租 · 滨江小区2居室-南,整租
2,30.37 ㎡06/6层1室1厅,徐汇,距4号线上海体育馆站236米,236,236.0,4690,30.37,30.37,整租 · 中原新村1居室-南,整租
3,49.15 ㎡01/6层2室1厅,徐汇,距3号线石龙路站243米,243,243.0,4790,49.15,49.15,整租 · 东泉小区2居室-南,整租
4,33.17 ㎡04/6层1室1厅,徐汇,距4号线东安路站488米,488,488.0,4490,33.17,33.17,整租 · 斜土路2200弄1居室-南,整租


---

# 去重

In [7]:
data = data.drop_duplicates()
data.shape

(14453, 10)

---

# 替换值

In [8]:
data['metro_num'] = data['metro_num'].replace(0,np.nan)
data['price'] = data['price'].replace(0,np.nan)
data['square'] = data['square'].replace(0,np.nan)

In [9]:
#data[data['metro_num']==np.nan]
data['price'] = pd.to_numeric(data['price'])

## 将按天付款的房源价格乘以30

In [10]:
data['mon_price'] = data['price'] 
data.loc[data['price']<600,'mon_price'] *= 30        # 关键句

In [11]:
data = data.dropna()

In [12]:
data.shape

(14034, 11)

---

# 输出到csv

In [13]:
data.to_csv('housing_shanghai.csv')

---

# 准备数据

In [88]:
housing = data.copy()

In [89]:
housing.shape

(14034, 11)

In [90]:
housing['mon_price'] = pd.to_numeric(housing['mon_price'])

In [91]:
housing = housing[['district','type','metro_num','S','mon_price']]
housing.head()

,district,type,metro_num,S,mon_price
0,徐汇,整租,173.0,42.00,4830
1,徐汇,整租,757.0,68.65,5130
2,徐汇,整租,236.0,30.37,4690
3,徐汇,整租,243.0,49.15,4790
4,徐汇,整租,488.0,33.17,4490


In [92]:
housing_num = housing[['metro_num','S','mon_price']]
housing_num.head()

,metro_num,S,mon_price
0,173.0,42.00,4830
1,757.0,68.65,5130
2,236.0,30.37,4690
3,243.0,49.15,4790
4,488.0,33.17,4490


In [93]:
housing_cat = housing[['district','type']]
housing_cat.head()

,district,type
0,徐汇,整租
1,徐汇,整租
2,徐汇,整租
3,徐汇,整租
4,徐汇,整租


---

# 流水线

## 特征缩放

In [94]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer

num_pipeline = Pipeline([
        ('imputer', Imputer(strategy="median")),
        ('std_scaler', StandardScaler())
    ])

C:\Users\13626\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [95]:
ss=StandardScaler()
housing_num_tr = ss.fit_transform(housing_num)
ss.inverse_transform(housing_num_tr,copy=True)[:5]

C:\Users\13626\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\13626\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


array([[ 173.  ,   42.  , 4830.  ],
       [ 757.  ,   68.65, 5130.  ],
       [ 236.  ,   30.37, 4690.  ],
       [ 243.  ,   49.15, 4790.  ],
       [ 488.  ,   33.17, 4490.  ]])

## 数值和类别的流水线

In [96]:
from sklearn.compose import ColumnTransformer

In [97]:
num_attribs = list(housing_num)
cat_attribs = ['district','type']

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing).toarray()

C:\Users\13626\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [98]:
housing_prepared

array([[-1.26863157,  1.14766383,  1.46682065, ...,  0.        ,
         0.        ,  1.        ],
       [-0.11263402,  2.55204439,  1.67270329, ...,  0.        ,
         0.        ,  1.        ],
       [-1.14392635,  0.53479532,  1.37074209, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.50061597, -0.5175679 , -0.64004503, ...,  0.        ,
         1.        ,  0.        ],
       [-0.3224555 , -1.06561885, -0.61945677, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.00811229, -0.6176926 , -0.77730012, ...,  0.        ,
         1.        ,  0.        ]])

---

# 聚类

In [99]:
from sklearn.cluster import KMeans # 导入Kmeans
k=5 # 聚类类别数
kmodel = KMeans(n_clusters=k, n_jobs=4)
kmodel.fit(housing_prepared)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=4, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

## 聚类结果的前三列

In [100]:
kmodel.cluster_centers_[0:5,:3]

array([[-0.31836645,  1.36617675,  1.26256451],
       [ 1.6756617 , -0.37700987, -0.61895916],
       [ 0.17679781, -0.42650547, -0.51328037],
       [-0.82708043, -0.40927334, -0.11960873],
       [-0.26395918,  3.19848615,  2.86549821]])

## 聚类结果反特征缩放

In [101]:
ss.inverse_transform(kmodel.cluster_centers_[0:5,:3],copy=True)

array([[ 653.06574924,   46.14657492, 4532.37003058],
       [1660.43159258,   13.06727665, 1790.72507553],
       [ 903.21846847,   12.12803153, 1944.71396396],
       [ 396.06783276,   12.45503413, 2518.34897611],
       [ 680.55183413,   80.91709729, 6868.07017544]])

分别为：距离地铁的距离，面积，价格

---

## 将聚类类别插入DataFrame

In [102]:
housing_data = housing.copy()
housing_data.head(1)

,district,type,metro_num,S,mon_price
0,徐汇,整租,173.0,42.0,4830


In [103]:
label = Series(kmodel.labels_,index=housing_data.index)

housing_data.insert(5,'聚类类别',label)
#kmeans_job

In [104]:
housing_data.head()

,district,type,metro_num,S,mon_price,聚类类别
0,徐汇,整租,173.0,42.00,4830,0
1,徐汇,整租,757.0,68.65,5130,0
2,徐汇,整租,236.0,30.37,4690,0
3,徐汇,整租,243.0,49.15,4790,0
4,徐汇,整租,488.0,33.17,4490,0


---

## 转换为房屋等级

In [105]:
c = pd.DataFrame(kmodel.cluster_centers_).sort_values(2)
c['房屋等级']=['交通不便普通','交通一般普通','交通便利普通','高端房源','豪宅']
#c

In [106]:
labels = c
# 高价值
expensive = labels[labels['房屋等级']=='豪宅'].index[0]
high = labels[labels['房屋等级']=='高端房源'].index[0]
mid = labels[labels['房屋等级']=='交通便利普通'].index[0]
normal = labels[labels['房屋等级']=='交通一般普通'].index[0]
low = labels[labels['房屋等级']=='交通不便普通'].index[0]
#print(expensive,high,mid,low)

In [107]:
kmeans_type_shift = {
    expensive:'豪宅',
    high:'高端房源',
    mid:'交通便利普通',
    normal:'交通一般普通',
    low:'交通不便普通'
}
housing_data['聚类类别'] = housing_data['聚类类别'].map(kmeans_type_shift)

In [108]:
housing_data.head()

,district,type,metro_num,S,mon_price,聚类类别
0,徐汇,整租,173.0,42.00,4830,高端房源
1,徐汇,整租,757.0,68.65,5130,高端房源
2,徐汇,整租,236.0,30.37,4690,高端房源
3,徐汇,整租,243.0,49.15,4790,高端房源
4,徐汇,整租,488.0,33.17,4490,高端房源


---

In [115]:
housing_data.shape
#housing_data.info()

(14034, 6)

---

# 聚类后的Groupby

In [261]:
grouped_house = housing_data[['metro_num','S','mon_price']].groupby(housing_data['聚类类别'])
#grouped_house

In [262]:
grouped_house.mean().sort_values(by='mon_price',ascending=True).round(2)

,metro_num,S,mon_price
聚类类别,,,
交通不便普通,1660.43,13.07,1790.73
交通一般普通,903.22,12.13,1944.71
交通便利普通,396.07,12.46,2518.35
高端房源,653.07,46.15,4532.37
豪宅,680.55,80.92,6868.07


---

# 分区

In [156]:
housing_data['district'].value_counts() # 各区房源总数

浦东    2220
宝山    1925
闵行    1883
嘉定    1569
松江    1565
普陀    1372
徐汇     864
杨浦     645
长宁     555
闸北     484
虹口     446
黄浦     270
青浦     182
静安      54
Name: district, dtype: int64

---

# 转化为全上海类别的DataFrame

In [310]:
low_type = housing_data[housing_data['聚类类别']=='交通不便普通']['district'].value_counts()
normal_type = housing_data[housing_data['聚类类别']=='交通一般普通']['district'].value_counts()
mid_type = housing_data[housing_data['聚类类别']=='交通便利普通']['district'].value_counts()
high_type = housing_data[housing_data['聚类类别']=='高端房源']['district'].value_counts()
expensive_type = housing_data[housing_data['聚类类别']=='豪宅']['district'].value_counts()

In [311]:
low_type = Series(low_type)
normal_type = Series(normal_type)
mid_type = Series(mid_type)
high_type = Series(high_type)
expensive_type = Series(expensive_type)

In [314]:
temp_pd = DataFrame(housing_data['district'].value_counts())

In [315]:
# '交通不便普通','交通一般普通','交通便利普通','高端房源','豪宅'
temp_pd.insert(1,'交通不便普通',low_type)
temp_pd.insert(2,'交通一般普通',normal_type)
temp_pd.insert(3,'交通便利普通',mid_type)
temp_pd.insert(4,'高端房源',high_type)
temp_pd.insert(5,'豪宅',expensive_type)

In [319]:
shanghai_housing = temp_pd.fillna(0).astype(int)

In [320]:
shanghai_housing

,district,交通不便普通,交通一般普通,交通便利普通,高端房源,豪宅
浦东,2220,164,777,762,414,103
宝山,1925,464,896,371,141,53
闵行,1883,283,848,516,186,50
嘉定,1569,624,479,242,166,58
松江,1565,405,641,403,82,34
普陀,1372,204,290,613,209,56
徐汇,864,32,42,541,179,70
杨浦,645,24,162,264,163,32
长宁,555,14,72,306,113,50
闸北,484,32,119,206,88,39


In [321]:
shanghai_housing.to_csv('shanghai_housing_type.csv')

---